In [1]:
import pandas as pd
import re
from fuzzywuzzy import process

In [2]:
inputpublicfile = 'data/student_admissions/public_admissions.csv'
inputprivatefile = 'data/student_admissions/private_admissions.csv'
outputfile = 'allschools.csv'

In [3]:
household_education = pd.read_csv('data/zip_data/household_education_level.csv')
household_income = pd.read_csv('data/zip_data/household_income_level.csv')
public_admissions = pd.read_csv(inputpublicfile)
private_admissions = pd.read_csv(inputprivatefile)
zcta_to_zip = pd.read_csv('data/zip_data/ZCTA_to_ZIP.csv')
high_schools = pd.read_csv('data/zip_data/high_schools.csv')

/var/folders/p0/cx2wcl6d7w79c__n36_zynk00000gn/T/ipykernel_17972/2465730649.py:1: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  household_education = pd.read_csv('data/zip_data/household_education_level.csv')


In [4]:
# Drop census columns for error and annotations
drop_cols_education = set(line.rstrip() for line in open('data/zip_data/drop_education.txt'))
drop_cols_income = set(line.rstrip() for line in open('data/zip_data/drop_income.txt'))

household_education = household_education.drop(columns=drop_cols_education)
household_income = household_income.drop(columns=drop_cols_income)

# pull in admissions data and pivot to get values for app/adm for each school
public_admissions = (public_admissions.loc[(public_admissions['Uad Uc Ethn 6 Cat'] == 'All') & 
                                           (public_admissions['Count'] != 'Enr'), 
                                           ['City', 'School name', 'Count', 'Pivot Field Values']])
private_admissions = (private_admissions.loc[(private_admissions['Uad Uc Ethn 6 Cat'] == 'All') &
                                             (private_admissions['Count'] != 'Enr'),
                                             ['City', 'School name', 'Count', 'Pivot Field Values']])

public_admissions = public_admissions.pivot(columns = 'Count', values = 'Pivot Field Values', index = ['City', 'School name']).reset_index()
private_admissions = private_admissions.pivot(columns = 'Count', values = 'Pivot Field Values', index = ['City', 'School name']).reset_index()

In [5]:
mapvals = {'high': '', 'school': '', 'hs': '', 'junior/senior ': ' ', 'senior ': ' '}

def clean_name(name):
    name = name.lower()
    for k, v in mapvals.items():
        name = name.replace(k, v)

    name = re.sub(r'\bacad\b', 'academy', name)

    return name

In [6]:
public_admissions['Match'] = public_admissions['School name'].apply(clean_name)
private_admissions['Match'] = private_admissions['School name'].apply(clean_name)
high_schools['Match'] = high_schools['School'].apply(clean_name)

In [7]:
public = high_schools.loc[high_schools['Public Yes/No'] == 'Y']
private = high_schools.loc[high_schools['Public Yes/No'] == 'N']

public_admissions['Match'] = public_admissions.apply(lambda x: next(iter(process.extractOne(x['Match'], public.loc[public['Street City'] == x['City']]['Match'], score_cutoff=80) or []), None), axis=1)
private_admissions['Match'] = private_admissions.apply(lambda x: next(iter(process.extractOne(x['Match'], private.loc[private['Street City'] == x['City']]['Match'], score_cutoff=80) or []), None), axis=1)

In [8]:
publicdf = pd.merge(public_admissions, high_schools, how='left', left_on=['Match', 'City'], right_on=['Match', 'Street City'])
privatedf = pd.merge(private_admissions, high_schools, how='left', left_on=['Match', 'City'], right_on=['Match', 'Street City'])

In [9]:
len(privatedf.loc[privatedf['Street City'].isna()])/len(privatedf)

0.11142857142857143

In [10]:
len(publicdf.loc[publicdf['Street City'].isna()])/len(publicdf)

0.04194260485651214

In [11]:
publicdf = publicdf.loc[publicdf['Match'].notna()]
privatedf = privatedf.loc[privatedf['Match'].notna()]

df = pd.concat([publicdf, privatedf])
df['Street Zip'] = df['Street Zip'].apply(lambda x: x[:5])

zcta_to_zip = zcta_to_zip[['ZIP_CODE', 'ZCTA']]

df = df[['School', 'City', 'App', 'Adm', 'Charter Yes/No', 'Public Yes/No', 'Street Zip']].astype({'Street Zip': int})
df = pd.merge(df, zcta_to_zip, how='left', left_on='Street Zip', right_on='ZIP_CODE')
df['Zip'] = df['ZCTA'].astype(int)
df = df.drop(['Street Zip', 'ZIP_CODE', 'ZCTA'], axis=1)

household_income['Geographic Area Name'] = household_income['Geographic Area Name'].apply(lambda x: int(x[5:]))
household_education['Geographic Area Name'] = household_education['Geographic Area Name'].apply(lambda x: int(x[5:]))
df = pd.merge(df, household_income, how='left', left_on='Zip', right_on='Geographic Area Name')
df = pd.merge(df, household_education, how='left', left_on='Zip', right_on='Geographic Area Name')

df.columns = df.columns.str.replace('Estimate!!Total:!!', '')
df.columns = df.columns.str.replace('Estimate!!Households!!Total!!', '')
df.columns = df.columns.str.replace('Estimate!!Households!!', '')

In [12]:
# schoolGroups = {'some': ['No schooling completed', 'Nursery school', 'Kindergarten'],
# 'elementary': ['1st grade', '2nd grade', '3rd grade', '4th grade', '5th grade'],
# 'middle': ['6th grade', '7th grade', '8th grade'],
# 'high': ['9th grade', '10th grade', '11th grade', '12th grade, no diploma', 'Regular high school diploma', 'GED or alternative credential'],
# 'higher': ['Some college, less than 1 year', 'Some college, 1 or more years, no degree', 'Associate\'s degree', 'Bachelor\'s degree'],
# 'grad': ['Master\'s degree', 'Professional school degree', 'Doctorate degree']}

schoolGroups = { 'none': ['No schooling completed'],
'high': ['12th grade, no diploma', 'Regular high school diploma', 'GED or alternative credential'],
'higher': ['Associate\'s degree', 'Bachelor\'s degree'],
'grad': ['Master\'s degree', 'Professional school degree', 'Doctorate degree']}

for groupname, grouplist in schoolGroups.items():
    df[groupname] = df[grouplist].sum(axis=1)/df['Estimate!!Total:']
    df.drop(grouplist, axis=1, inplace=True)

# schoolGroups = ['No schooling completed', 'Nursery school', 'Kindergarten',
#                 '1st grade', '2nd grade', '3rd grade', '4th grade', '5th grade',
#                 '6th grade', '7th grade', '8th grade',
#                 '9th grade', '10th grade', '11th grade', '12th grade, no diploma', 'Regular high school diploma', 'GED or alternative credential',
#                 'Some college, less than 1 year', 'Some college, 1 or more years, no degree', 'Associate\'s degree', 'Bachelor\'s degree', 
#                 'Master\'s degree', 'Professional school degree', 'Doctorate degree']

# for group in schoolGroups:
#     df[group] = df[group]/df['Estimate!!Total:']

df['Public Yes/No'] = df['Public Yes/No'].map({'Y': 1, 'N':0})
df['Charter Yes/No'] = df['Charter Yes/No'].map({'Y': 1, 'N':0})

In [13]:
keeplist = ['School', 'Public Yes/No', 'Charter Yes/No', 'App', 'Adm', 'Median income (dollars)', 'Total'] + list(schoolGroups.keys())
#keeplist = ['School', 'Public Yes/No', 'Charter Yes/No', 'App', 'Adm', 'Median income (dollars)', 'Total'] + schoolGroups

In [14]:
df = df.loc[df['Median income (dollars)'] != '-']
df.loc[df['Median income (dollars)'] == '250,000+', 'Median income (dollars)'] = 250000
df[keeplist].to_csv('data/merged/{}'.format(outputfile))

In [15]:
len(public_admissions) + len(private_admissions)

1706

In [16]:
len(df)

1608